Sumber: https://towardsdatascience.com/feature-selection-in-machine-learning-using-lasso-regression-7809c7c2771a

kenapa harus di scaling? karena kalo ga di scaling dia modelnya gak nemukan fitur dengan nilai importance yang lebih dari 0

In [2]:
#libraries buat seleksi fitur
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso

#libraries buat klasifikasi
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [3]:
#import dataset
df = pd.read_csv('nirsMangga.csv')

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(labels=['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis=1),
    df['label'],
    test_size=0.3,
    random_state=0)

x_train.head()

,999.9,1000.3,1000.7,1001.1,1001.4,1001.8,1002.2,1002.6,1003,1003.4,...,2478.7,2481.1,2483.5,2485.8,2488.2,2490.6,2493,2495.4,2497.8,2500.2
16,0.471459,0.471074,0.470934,0.470379,0.470260,0.469880,0.469497,0.469435,0.469454,0.468998,...,1.413537,1.415740,1.417568,1.419698,1.421711,1.423070,1.424394,1.426121,1.427552,1.428625
51,0.433239,0.432622,0.432626,0.432379,0.431620,0.430710,0.430836,0.430847,0.430188,0.429470,...,1.601232,1.602877,1.604524,1.605982,1.606778,1.607837,1.608756,1.609967,1.610900,1.611099
183,0.545045,0.544204,0.543792,0.543596,0.543338,0.542534,0.541493,0.541139,0.541308,0.540831,...,1.524657,1.525973,1.527454,1.529518,1.530097,1.530315,1.530254,1.531191,1.532366,1.533183
145,0.545846,0.544815,0.544524,0.544631,0.544169,0.543143,0.542535,0.542080,0.541842,0.541258,...,1.421962,1.422955,1.423717,1.424639,1.425080,1.425797,1.426503,1.427164,1.427838,1.428271
40,0.381048,0.380483,0.380541,0.380151,0.379599,0.379189,0.379009,0.378722,0.378309,0.377719,...,1.571125,1.572674,1.574303,1.576075,1.577273,1.577980,1.578561,1.579334,1.580042,1.581424


In [4]:
#buat dataframe x (data input) dan y (data output)
x = df.drop(labels=['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis=1)
y = df['label']

#buat list nama fitur
features = x.columns.tolist()

# Seleksi Fitur LASSO

Since our dataset needs to be scaled in advance, we can make use of the powerful Pipeline object in scikit-learn. 

Our pipeline is made by a StandardScaler and the Lasso object itself.

In [5]:
%%time

#Bikin pipeline 
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

CPU times: user 89 µs, sys: 85 µs, total: 174 µs
Wall time: 202 µs


Now we have to optimize the α hyperparameter of Lasso regression. 

For this example, we are going to test several values from 0.1 to 10 with 0.1 step. 

For each value, we calculate the average value of the mean squared error in a 5-folds cross-validation and select the value of α that minimizes such average performance metrics. We can use the GridSearchCV object for this purpose.

In [6]:
%%time

#define model Lasso Regression yang akan dilatih
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

CPU times: user 60 µs, sys: 57 µs, total: 117 µs
Wall time: 128 µs


We use neg_mean_squared_error because the grid search tries to maximize the performance metrics, so we add a minus sign to minimize the mean squared error.

In [36]:
%%time

search.fit(x_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.767 total time=   0.3s
[CV 2/5] END .................model__alpha=0.1;, score=-0.445 total time=   0.3s
[CV 3/5] END .................model__alpha=0.1;, score=-1.074 total time=   0.3s
[CV 4/5] END .................model__alpha=0.1;, score=-0.454 total time=   0.2s
[CV 5/5] END .................model__alpha=0.1;, score=-0.481 total time=   0.4s
[CV 1/5] END .................model__alpha=0.2;, score=-0.804 total time=   0.3s
[CV 2/5] END .................model__alpha=0.2;, score=-0.418 total time=   0.2s
[CV 3/5] END .................model__alpha=0.2;, score=-1.110 total time=   0.4s
[CV 4/5] END .................model__alpha=0.2;, score=-0.499 total time=   0.4s
[CV 5/5] END .................model__alpha=0.2;, score=-0.433 total time=   0.2s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.849 total time=   0.4s
[CV 2/5] END .model__alpha=0.30000000000000004;

[CV 2/5] END .................model__alpha=2.1;, score=-0.756 total time=   0.3s
[CV 3/5] END .................model__alpha=2.1;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=2.1;, score=-0.707 total time=   0.2s
[CV 5/5] END .................model__alpha=2.1;, score=-0.654 total time=   0.2s
[CV 1/5] END .................model__alpha=2.2;, score=-0.965 total time=   0.2s
[CV 2/5] END .................model__alpha=2.2;, score=-0.756 total time=   0.2s
[CV 3/5] END .................model__alpha=2.2;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=2.2;, score=-0.707 total time=   0.2s
[CV 5/5] END .................model__alpha=2.2;, score=-0.654 total time=   0.2s
[CV 1/5] END ..model__alpha=2.3000000000000003;, score=-0.965 total time=   0.2s
[CV 2/5] END ..model__alpha=2.3000000000000003;, score=-0.756 total time=   0.2s
[CV 3/5] END ..model__alpha=2.3000000000000003;, score=-1.246 total time=   0.1s
[CV 4/5] END ..model__alpha=

[CV 4/5] END .................model__alpha=4.1;, score=-0.707 total time=   0.2s
[CV 5/5] END .................model__alpha=4.1;, score=-0.654 total time=   0.2s
[CV 1/5] END .................model__alpha=4.2;, score=-0.965 total time=   0.3s
[CV 2/5] END .................model__alpha=4.2;, score=-0.756 total time=   0.1s
[CV 3/5] END .................model__alpha=4.2;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=4.2;, score=-0.707 total time=   0.2s
[CV 5/5] END .................model__alpha=4.2;, score=-0.654 total time=   0.2s
[CV 1/5] END .................model__alpha=4.3;, score=-0.965 total time=   0.2s
[CV 2/5] END .................model__alpha=4.3;, score=-0.756 total time=   0.2s
[CV 3/5] END .................model__alpha=4.3;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=4.3;, score=-0.707 total time=   0.2s
[CV 5/5] END .................model__alpha=4.3;, score=-0.654 total time=   0.2s
[CV 1/5] END ..model__alpha=

[CV 1/5] END .................model__alpha=6.2;, score=-0.965 total time=   0.2s
[CV 2/5] END .................model__alpha=6.2;, score=-0.756 total time=   0.2s
[CV 3/5] END .................model__alpha=6.2;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=6.2;, score=-0.707 total time=   0.3s
[CV 5/5] END .................model__alpha=6.2;, score=-0.654 total time=   0.2s
[CV 1/5] END .................model__alpha=6.3;, score=-0.965 total time=   0.2s
[CV 2/5] END .................model__alpha=6.3;, score=-0.756 total time=   0.3s
[CV 3/5] END .................model__alpha=6.3;, score=-1.246 total time=   0.1s
[CV 4/5] END .................model__alpha=6.3;, score=-0.707 total time=   0.2s
[CV 5/5] END .................model__alpha=6.3;, score=-0.654 total time=   0.2s
[CV 1/5] END .................model__alpha=6.4;, score=-0.965 total time=   0.2s
[CV 2/5] END .................model__alpha=6.4;, score=-0.756 total time=   0.2s
[CV 3/5] END ...............

[CV 3/5] END .................model__alpha=8.2;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=8.2;, score=-0.707 total time=   0.3s
[CV 5/5] END .................model__alpha=8.2;, score=-0.654 total time=   0.1s
[CV 1/5] END .................model__alpha=8.3;, score=-0.965 total time=   0.3s
[CV 2/5] END .................model__alpha=8.3;, score=-0.756 total time=   0.1s
[CV 3/5] END .................model__alpha=8.3;, score=-1.246 total time=   0.2s
[CV 4/5] END .................model__alpha=8.3;, score=-0.707 total time=   0.3s
[CV 5/5] END .................model__alpha=8.3;, score=-0.654 total time=   0.1s
[CV 1/5] END .................model__alpha=8.4;, score=-0.965 total time=   0.1s
[CV 2/5] END .................model__alpha=8.4;, score=-0.756 total time=   0.2s
[CV 3/5] END .................model__alpha=8.4;, score=-1.246 total time=   0.1s
[CV 4/5] END .................model__alpha=8.4;, score=-0.707 total time=   0.2s
[CV 5/5] END ...............

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [37]:
search.best_params_

{'model__alpha': 0.1}

In [38]:
#manggil coef setiap fitur
coefficients = search.best_estimator_.named_steps['model'].coef_
importance = np.abs(coefficients)

##### Features yang diambil

In [39]:
np.array(features)[importance > 0]

array(['1323.5', '1326.2', '1326.9', '1328.9', '1331', '1331.7', '1337.1',
       '1397.7', '1398.5'], dtype='<U6')

##### Features yang ilang

In [40]:
np.array(features)[importance == 0]

array(['999.9', '1000.3', '1000.7', ..., '2495.4', '2497.8', '2500.2'],
      dtype='<U6')

# Klasifikasi Random Forest Classifier

In [41]:
#buat n-fold cv
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

## Testing Performa Model Random Forest Classifier

In [42]:
%%time

#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
n_feat = [9]
n_trees = [100]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur LASSO
        x_train_selected = x_train[np.array(features)[importance > 0]]
        x_test_selected = x_test[np.array(features)[importance > 0]]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RF data Train dengan 9 fitur dan 100 trees:1.0
Akurasi model RF data Test dengan 9 fitur dan 100 trees:0.88
Precision model RF data Train dengan 9 fitur dan 100 trees:1.0
Precision model RF data Test dengan 9 fitur dan 100 trees:0.88
Recall model RF data Train dengan 9 fitur dan 100 trees:1.0
Recall model RF data Test dengan 9 fitur dan 100 trees:0.88
 
CPU times: user 1.52 s, sys: 8.94 ms, total: 1.53 s
Wall time: 1.53 s


In [28]:
%%time


#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
n_feat = [9]
n_trees = [150]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur LASSO
        x_train_selected = x_train[np.array(features)[importance > 0]]
        x_test_selected = x_test[np.array(features)[importance > 0]]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RF data Train dengan 9 fitur dan 150 trees:1.0
Akurasi model RF data Test dengan 9 fitur dan 150 trees:0.89
Precision model RF data Train dengan 9 fitur dan 150 trees:1.0
Precision model RF data Test dengan 9 fitur dan 150 trees:0.89
Recall model RF data Train dengan 9 fitur dan 150 trees:1.0
Recall model RF data Test dengan 9 fitur dan 150 trees:0.89
 
CPU times: user 2.41 s, sys: 26.8 ms, total: 2.44 s
Wall time: 2.44 s


In [29]:
%%time


#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
n_feat = [9]
n_trees = [200]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur LASSO
        x_train_selected = x_train[np.array(features)[importance > 0]]
        x_test_selected = x_test[np.array(features)[importance > 0]]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RF data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RF data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RF data Train dengan 9 fitur dan 200 trees:1.0
Akurasi model RF data Test dengan 9 fitur dan 200 trees:0.91
Precision model RF data Train dengan 9 fitur dan 200 trees:1.0
Precision model RF data Test dengan 9 fitur dan 200 trees:0.91
Recall model RF data Train dengan 9 fitur dan 200 trees:1.0
Recall model RF data Test dengan 9 fitur dan 200 trees:0.91
 
CPU times: user 3.18 s, sys: 37.7 ms, total: 3.21 s
Wall time: 3.22 s


# Klasifikasi Support Vector Machine

Untuk train dan test set, serta cv-nya ambil dari yang Random Forest Classifier, sama aja 

## Testing Performa Model Support Vector Machine

In [30]:
%%time



#tentukan metode scoring yang digunakan
scoring_svm = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur yang digunakan dalam proses klasifikasi SVM ini
n_feat = [9]

for nfeat in n_feat:     
    #ambil n fitur input hasil seleksi fitur LASSO
    x_train_selected = x_train[np.array(features)[importance > 0]]
    x_test_selected = x_test[np.array(features)[importance > 0]]

    #Create a Support Vector Classifier
    clf_svm = svm.SVC()

    #Train the model using the training sets
    clf_svm.fit(x_train_selected, y_train)
    y_pred_svm = clf_svm.predict(x_test_selected)

    #hitung score model dari data train
    scores_svm = cross_validate(clf_svm, x_train_selected, y_train, scoring=scoring_svm, cv=cv, return_train_score=True)

    print("akurasi model SVM data Train dengan " + str(nfeat) + " fitur: " 
          + str(round(scores_svm['train_acc'].mean(), 2)))
    print("akurasi model SVM data Test dengan " + str(nfeat) + " fitur: " 
          + str(round(metrics.accuracy_score(y_test, y_pred_svm), 2)))
    print("Precision model SVM data Train dengan " + str(nfeat) + " fitur:"
          + str(round(scores_svm['train_prec_micro'].mean(), 2)))
    print("Precision model SVM data Test dengan " + str(nfeat) + " fitur:"
          + str(round(metrics.precision_score(y_test, y_pred_svm, average='micro'), 2)))
    print("Recall model SVM data Train dengan " + str(nfeat) + " fitur:"
          + str(round(scores_svm['train_rec_micro'].mean(), 2)))
    print("Recall model SVM data Test dengan " + str(nfeat) + " fitur:"
          + str(round(metrics.recall_score(y_test, y_pred_svm, average='micro'), 2)))
    print(" ")

akurasi model SVM data Train dengan 9 fitur: 0.77
akurasi model SVM data Test dengan 9 fitur: 0.66
Precision model SVM data Train dengan 9 fitur:0.77
Precision model SVM data Test dengan 9 fitur:0.66
Recall model SVM data Train dengan 9 fitur:0.77
Recall model SVM data Test dengan 9 fitur:0.66
 
CPU times: user 104 ms, sys: 2.9 ms, total: 107 ms
Wall time: 105 ms


# Misc

In [13]:
#ambil semua fitur yang penting (nilai koefisien tidak bernilai 0)
# x_train_selected = X_train.drop(labels=np.array(x_features)[coef == 0], axis=1)
# x_test_selected = x_test.drop(labels=np.array(x_features)[coef == 0], axis=1)

#ambil n fitur
x_train_selected = x_train[np.array(features)[importance > 0]]
x_test_selected = x_test[np.array(features)[importance > 0]]
len(x_train_selected.sum())

9